# H4 Custom Reform Analysis

This notebook implements SSA's H4 proposal using PolicyEngine's parameter system:

**H4: Increase the threshold for taxation of OASDI benefits to $50,000 for single filers and $100,000 for joint filers starting in 2027**

## Reform Details

Under current law (as of 2027):
- **Base threshold** (50% taxation): $25,000 (single), $32,000 (joint)
- **Additional threshold** (85% taxation): $34,000 (single), $44,000 (joint)

Under H4 reform:
- **Base threshold**: $50,000 (single), $100,000 (joint)
- **Additional threshold**: $50,000 (single), $100,000 (joint)

This effectively sets both thresholds to the same amount, meaning:
- Below threshold: 0% of SS benefits are taxable
- Above threshold: 85% of SS benefits are taxable (no 50% tier)

In [1]:
# Import libraries
import sys
import os
from pathlib import Path

# Add project root to path
project_root = Path.cwd().parent
sys.path.insert(0, str(project_root / 'src'))

import pandas as pd
import numpy as np
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from policyengine_us import Microsimulation
from policyengine_core.reforms import Reform
from IPython.display import HTML, display
import warnings
warnings.filterwarnings('ignore')

print("✓ Libraries imported successfully")
print(f"✓ Project root: {project_root}")

c:\Users\dtsax\envs\pe\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


✓ Libraries imported successfully
✓ Project root: c:\Users\dtsax\PolicyEngine\crfb-tob-impacts


## 1. Define H4 Reform

Create a reform that modifies the Social Security taxability thresholds starting in 2027.

In [2]:
def create_h4_reform():
    """
    Create H4 reform: Increase SS taxability thresholds to $50k/$100k.
    
    Sets both base and additional thresholds to:
    - $50,000 for single, separate, head of household, and surviving spouse filers
    - $100,000 for joint filers
    - Starting in 2027-01-01
    """
    def modify(parameters, period, _):
        # Modify base threshold
        base_threshold = parameters.gov.irs.social_security.taxability.threshold.base.main
        base_threshold.SINGLE.update(period=period, value=50_000)
        base_threshold.JOINT.update(period=period, value=100_000)
        base_threshold.SEPARATE.update(period=period, value=50_000)
        base_threshold.HEAD_OF_HOUSEHOLD.update(period=period, value=50_000)
        base_threshold.SURVIVING_SPOUSE.update(period=period, value=50_000)
        
        # Modify additional threshold
        additional_threshold = parameters.gov.irs.social_security.taxability.threshold.adjusted_base.main
        additional_threshold.SINGLE.update(period=period, value=50_000)
        additional_threshold.JOINT.update(period=period, value=100_000)
        additional_threshold.SEPARATE.update(period=period, value=50_000)
        additional_threshold.HEAD_OF_HOUSEHOLD.update(period=period, value=50_000)
        additional_threshold.SURVIVING_SPOUSE.update(period=period, value=50_000)
        
        return parameters
    
    return Reform.from_dict(
        {
            "gov.irs.social_security.taxability.threshold.base.main": {
                "2027-01-01.SINGLE": 50_000,
                "2027-01-01.JOINT": 100_000,
                "2027-01-01.SEPARATE": 50_000,
                "2027-01-01.HEAD_OF_HOUSEHOLD": 50_000,
                "2027-01-01.SURVIVING_SPOUSE": 50_000,
            },
            "gov.irs.social_security.taxability.threshold.adjusted_base.main": {
                "2027-01-01.SINGLE": 50_000,
                "2027-01-01.JOINT": 100_000,
                "2027-01-01.SEPARATE": 50_000,
                "2027-01-01.HEAD_OF_HOUSEHOLD": 50_000,
                "2027-01-01.SURVIVING_SPOUSE": 50_000,
            }
        },
        country_id="us"
    )

h4_reform = create_h4_reform()
print("✓ H4 reform created successfully")
print("\nReform parameters:")
print("  Base threshold - Single: $50,000")
print("  Base threshold - Joint: $100,000")
print("  Additional threshold - Single: $50,000")
print("  Additional threshold - Joint: $100,000")
print("  Effective date: 2027-01-01")

✓ H4 reform created successfully

Reform parameters:
  Base threshold - Single: $50,000
  Base threshold - Joint: $100,000
  Additional threshold - Single: $50,000
  Additional threshold - Joint: $100,000
  Effective date: 2027-01-01


## 2. Calculate Revenue Impacts

Calculate the revenue impact of H4 reform for key years to compare against SSA benchmarks.

In [3]:
# Years to analyze (focus on SSA benchmark years)
analysis_years = [2030, 2040, 2050, 2060, 2070, 2080, 2090, 2100]

print("="*80)
print("CALCULATING H4 REVENUE IMPACTS")
print("="*80)

h4_results = []

for year in analysis_years:
    print(f"\nProcessing {year}...")
    
    # Load dataset
    dataset_path = f'hf://policyengine/test/{year}.h5'
    print(f"  Loading dataset: {dataset_path}")
    
    # Baseline simulation
    print(f"  Running baseline simulation...")
    baseline = Microsimulation(dataset=dataset_path)
    baseline_revenue = baseline.calculate("income_tax").sum()
    
    # Reform simulation
    print(f"  Running H4 reform simulation...")
    reform_sim = Microsimulation(dataset=dataset_path, reform=h4_reform)
    reform_revenue = reform_sim.calculate("income_tax").sum()
    
    # Calculate impact
    revenue_impact = reform_revenue - baseline_revenue
    
    # Calculate taxable payroll
    print(f"  Computing taxable payroll...")
    taxable_ss_earnings = baseline.calculate("taxable_earnings_for_social_security")
    taxable_self_employment = baseline.calculate("social_security_taxable_self_employment_income")
    total_taxable_payroll = taxable_ss_earnings.sum() + taxable_self_employment.sum()
    
    # Calculate SS benefits
    print(f"  Computing SS benefits...")
    ss_benefits = baseline.calculate("social_security")
    total_ss_benefits = ss_benefits.sum()
    
    h4_results.append({
        'year': year,
        'baseline_revenue_billions': baseline_revenue / 1e9,
        'reform_revenue_billions': reform_revenue / 1e9,
        'revenue_impact_billions': revenue_impact / 1e9,
        'taxable_payroll_billions': total_taxable_payroll / 1e9,
        'ss_benefits_billions': total_ss_benefits / 1e9,
    })
    
    print(f"  ✓ Baseline Revenue: ${baseline_revenue / 1e9:,.2f}B")
    print(f"  ✓ Reform Revenue: ${reform_revenue / 1e9:,.2f}B")
    print(f"  ✓ Revenue Impact: ${revenue_impact / 1e9:,.2f}B")
    print(f"  ✓ Taxable Payroll: ${total_taxable_payroll / 1e9:,.2f}B")
    print(f"  ✓ SS Benefits: ${total_ss_benefits / 1e9:,.2f}B")

h4_df = pd.DataFrame(h4_results)

print("\n" + "="*80)
print("✓ H4 revenue impact calculation complete")
print("="*80)

CALCULATING H4 REVENUE IMPACTS

Processing 2030...
  Loading dataset: hf://policyengine/test/2030.h5
  Running baseline simulation...
  Running H4 reform simulation...


ValueError: 'SINGLE' is not a valid instant. Instants are described using the 'YYYY-MM-DD' format, for instance '2015-06-15'.

## 3. Load SSA Benchmark Data

Load SSA's H4 estimates and projected taxable payroll for comparison.

In [ ]:
# SSA projected taxable payroll from 2024 Trustees Report Table VI.G6
ssa_projected_payroll = {
    2030: 17_803,
    2040: 27_337,
    2050: 34_419,
    2060: 41_836,
    2070: 50_887,
    2080: 79_177,
    2090: 126_652,
    2100: 191_588,
}

# SSA H4 estimates (change in annual balance as % of taxable payroll)
# Source: https://www.ssa.gov/oact/solvency/provisions/tables/table_run131.html
ssa_h4_pct_payroll = {
    2030: None,  # TODO: Add from SSA table
    2040: None,  # TODO: Add from SSA table  
    2050: None,  # TODO: Add from SSA table
    2060: -0.07,
    2070: None,  # TODO: Add from SSA table
    2080: None,  # TODO: Add from SSA table
    2090: None,  # TODO: Add from SSA table
    2100: -0.01,
}

# Add SSA data to results
h4_df['ssa_payroll_billions'] = h4_df['year'].map(ssa_projected_payroll)
h4_df['ssa_h4_pct_payroll'] = h4_df['year'].map(ssa_h4_pct_payroll)
h4_df['ssa_h4_revenue_billions'] = (
    h4_df['ssa_h4_pct_payroll'] / 100 * h4_df['ssa_payroll_billions']
)

# Calculate PE impact as % of SSA taxable payroll
h4_df['pe_pct_ssa_payroll'] = (
    h4_df['revenue_impact_billions'] / h4_df['ssa_payroll_billions'] * 100
)

# Calculate difference
h4_df['diff_pct_payroll'] = (
    h4_df['pe_pct_ssa_payroll'] - h4_df['ssa_h4_pct_payroll']
)
h4_df['diff_revenue_billions'] = (
    h4_df['revenue_impact_billions'] - h4_df['ssa_h4_revenue_billions']
)

print("✓ SSA benchmark data loaded")
print(f"\nSSA projected payroll range: ${ssa_projected_payroll[2030]:,}B (2030) to ${ssa_projected_payroll[2100]:,}B (2100)")

## 4. Comparison Table

Compare PolicyEngine H4 custom reform with SSA H4 benchmark.

In [ ]:
print("\n" + "="*80)
print("COMPARISON: PolicyEngine H4 Reform vs SSA H4 Benchmark")
print("="*80)
print("\nAll revenue figures in billions ($B)")
print("Percentages shown as % of SSA projected taxable payroll\n")

display_df = h4_df[[
    'year',
    'ss_benefits_billions',
    'taxable_payroll_billions',
    'ssa_payroll_billions',
    'revenue_impact_billions',
    'ssa_h4_revenue_billions',
    'diff_revenue_billions',
    'pe_pct_ssa_payroll',
    'ssa_h4_pct_payroll',
    'diff_pct_payroll'
]].copy()

# Format column names
display_df.columns = [
    'Year',
    'SS Benefits ($B)',
    'PE Taxable Payroll ($B)',
    'SSA Taxable Payroll ($B)',
    'PE H4 Impact ($B)',
    'SSA H4 Impact ($B)',
    'Difference ($B)',
    'PE (% SSA Payroll)',
    'SSA (% Payroll)',
    'Diff (pp)'
]

print(display_df.to_string(index=False))

## 5. Visualization

Visualize the comparison between PolicyEngine H4 reform and SSA H4 benchmark.

In [ ]:
# Create comparison visualizations
fig = make_subplots(
    rows=2, cols=2,
    subplot_titles=(
        'Revenue Impact: PE H4 vs SSA H4',
        'Revenue Impact as % of SSA Payroll',
        'Total Social Security Benefits',
        'Taxable Payroll: PE vs SSA Projections'
    ),
    vertical_spacing=0.15,
    horizontal_spacing=0.12
)

years = h4_df['year']
ssa_data = h4_df[h4_df['ssa_h4_revenue_billions'].notna()]

# 1. Revenue Impact (Dollar amounts)
fig.add_trace(
    go.Scatter(
        x=years,
        y=h4_df['revenue_impact_billions'],
        mode='lines+markers',
        name='PolicyEngine H4',
        line=dict(color='#2E86AB', width=3),
        marker=dict(size=8),
        text=h4_df['revenue_impact_billions'].apply(lambda x: f"${x:.1f}B"),
        hovertemplate='<b>Year:</b> %{x}<br><b>PE H4:</b> %{text}<extra></extra>'
    ),
    row=1, col=1
)

if len(ssa_data) > 0:
    fig.add_trace(
        go.Scatter(
            x=ssa_data['year'],
            y=ssa_data['ssa_h4_revenue_billions'],
            mode='markers',
            name='SSA H4 Benchmark',
            marker=dict(color='#A23B72', size=12, symbol='diamond'),
            text=ssa_data['ssa_h4_revenue_billions'].apply(lambda x: f"${x:.1f}B"),
            hovertemplate='<b>Year:</b> %{x}<br><b>SSA H4:</b> %{text}<extra></extra>'
        ),
        row=1, col=1
    )

# 2. Revenue Impact (% of payroll)
fig.add_trace(
    go.Scatter(
        x=years,
        y=h4_df['pe_pct_ssa_payroll'],
        mode='lines+markers',
        name='PolicyEngine H4',
        line=dict(color='#2E86AB', width=3),
        marker=dict(size=8),
        showlegend=False,
        text=h4_df['pe_pct_ssa_payroll'].apply(lambda x: f"{x:.3f}%"),
        hovertemplate='<b>Year:</b> %{x}<br><b>PE:</b> %{text}<extra></extra>'
    ),
    row=1, col=2
)

if len(ssa_data) > 0:
    fig.add_trace(
        go.Scatter(
            x=ssa_data['year'],
            y=ssa_data['ssa_h4_pct_payroll'],
            mode='markers',
            name='SSA H4',
            marker=dict(color='#A23B72', size=12, symbol='diamond'),
            showlegend=False,
            text=ssa_data['ssa_h4_pct_payroll'].apply(lambda x: f"{x:.3f}%"),
            hovertemplate='<b>Year:</b> %{x}<br><b>SSA:</b> %{text}<extra></extra>'
        ),
        row=1, col=2
    )

# 3. Total SS Benefits
fig.add_trace(
    go.Scatter(
        x=years,
        y=h4_df['ss_benefits_billions'],
        mode='lines+markers',
        name='SS Benefits',
        line=dict(color='#06BEE1', width=3),
        marker=dict(size=8),
        showlegend=False,
        text=h4_df['ss_benefits_billions'].apply(lambda x: f"${x:,.0f}B"),
        hovertemplate='<b>Year:</b> %{x}<br><b>Benefits:</b> %{text}<extra></extra>'
    ),
    row=2, col=1
)

# 4. Taxable Payroll comparison
fig.add_trace(
    go.Scatter(
        x=years,
        y=h4_df['taxable_payroll_billions'],
        mode='lines+markers',
        name='PolicyEngine',
        line=dict(color='#2E86AB', width=3),
        marker=dict(size=8),
        showlegend=False
    ),
    row=2, col=2
)

fig.add_trace(
    go.Scatter(
        x=years,
        y=h4_df['ssa_payroll_billions'],
        mode='lines+markers',
        name='SSA Projection',
        line=dict(color='#F18F01', width=3, dash='dash'),
        marker=dict(size=8),
        showlegend=True
    ),
    row=2, col=2
)

# Update axes
fig.update_yaxes(title_text="Revenue Impact ($B)", row=1, col=1)
fig.update_yaxes(title_text="% of SSA Payroll", row=1, col=2)
fig.update_yaxes(title_text="Billions ($)", row=2, col=1)
fig.update_yaxes(title_text="Taxable Payroll ($B)", row=2, col=2)

for i in range(1, 3):
    for j in range(1, 3):
        fig.update_xaxes(title_text="Year", row=i, col=j)

# Update layout
fig.update_layout(
    height=900,
    title_text="H4 Custom Reform Analysis: PolicyEngine vs SSA Benchmark<br><sub>Increase SS tax thresholds to $50k/$100k starting 2027</sub>",
    showlegend=True,
    legend=dict(
        orientation="h",
        yanchor="bottom",
        y=1.02,
        xanchor="right",
        x=1
    )
)

fig.show()

## 6. Export Results

In [ ]:
# Create output directory if it doesn't exist
output_dir = project_root / 'data'
output_dir.mkdir(exist_ok=True)

# Export H4 comparison
output_file = output_dir / 'h4_custom_reform_comparison.csv'
h4_df.to_csv(output_file, index=False)
print(f"✓ H4 reform comparison exported to: {output_file}")

# Display summary
print("\n" + "="*80)
print("KEY FINDINGS")
print("="*80)

ssa_years = h4_df[h4_df['ssa_h4_revenue_billions'].notna()]
for _, row in ssa_years.iterrows():
    print(f"\nYear {row['year']}:")
    print(f"  PolicyEngine H4: ${row['revenue_impact_billions']:.2f}B ({row['pe_pct_ssa_payroll']:.3f}% of SSA payroll)")
    print(f"  SSA H4 Benchmark: ${row['ssa_h4_revenue_billions']:.2f}B ({row['ssa_h4_pct_payroll']:.3f}% of SSA payroll)")
    print(f"  Difference: ${row['diff_revenue_billions']:+.2f}B ({row['diff_pct_payroll']:+.3f} percentage points)")
    print(f"  SSA projects: ${row['ssa_payroll_billions']:,.0f}B taxable payroll")
    print(f"  PolicyEngine: ${row['taxable_payroll_billions']:,.2f}B taxable payroll")

print("\n✓ H4 custom reform analysis complete!")